# Phase One Project UPDATE

### Where to open a new flower shop... in Silver Spring, Maryland or Leesburg, Virginia?

## Imports

In [29]:
import requests
import json
import pandas as pd
import numpy as np

## Yelp API Call

In [4]:
with open('api_key.txt') as f:
    lines = f.read()
 #   print(lines)

In [5]:
url = 'https://api.yelp.com/v3/businesses/search'
key = lines
headers = {
    'Authorization': 'Bearer %s' % key
}

In [6]:
data = []
for offset in range(0, 1000, 50):
    url_params = {
        'term': "flower",
        'location': 'Silver Spring+MD',
        'limit': 50,
        'offset': offset,
        "radius": 8047
    }

    response = requests.get(url, headers=headers, params=url_params)
    if response.status_code == 200:
        data += response.json()['businesses']

    elif response.status_code == 400:
            print('400 Bad Request')
            break


In [7]:
data2 = []
for offset in range(0, 1000, 50):
    url_params = {
        'term': "flower",
        'location': 'Leesburg+VA',
        'limit': 50,
        'offset': offset,
        "radius": 8047
    }

    response = requests.get(url, headers=headers, params=url_params)
    if response.status_code == 200:
        data2 += response.json()['businesses']

    elif response.status_code == 400:
            print('400 Bad Request')
            break


## Clean Yelp Results

In [8]:
def getbiz(params):
    names = []
    ratings = []
    pricing = []
    cnt = []
    address = []
    is_closed = []
    review_count = []
    categories = []
    city = []
    distance = []
    longitude = []
    latitude = []

    for i in range(len(params)):
        names.append(params[i]['name'])
        ratings.append(params[i]['rating'])
        address.append((params[i]['location']['address1']))
        is_closed.append(params[i]['is_closed'])
        review_count.append(params[i]['review_count'])
        categories.append(params[i]['categories'])
        city.append(params[i]['location']['city'])
        distance.append(params[i]['distance'])
        longitude.append(params[i]['coordinates']['longitude'])
        latitude.append(params[i]['coordinates']['latitude'])
        try:
            [pricing.append(len(params[i]['price']))]
        except:
            pricing.append(None)
        cnt.append(params[i]['review_count'])
    biz = {'name': names, 'rating': ratings, 'pricing': pricing, 'num_reviews': cnt, 'street_address': address, 
            'is_closed': is_closed, 'review_count': review_count, 'categories': categories, 'city': city,
            'distance': distance, 'longitude': longitude, 'latitude': latitude}


    return pd.DataFrame(biz)

In [9]:
ssdf = getbiz(data)
lbdf = getbiz(data2)

In [10]:
# Delete shops that are not within the respective cities
ssdf = ssdf[ssdf.city == 'Silver Spring']
lbdf = lbdf[lbdf.city == 'Leesburg']

In [12]:
ssdf.reset_index(inplace = True)
ssdf.drop('index', 1, inplace= True)

In [13]:
lbdf.reset_index(inplace = True)
lbdf.drop('index', 1, inplace= True)

In [14]:
#Save cleaned data to CSV
ssdf.to_csv('silver_spring.csv', encoding='utf-8', index=False)

lbdf.to_csv('leesburg.csv', encoding='utf-8', index=False)

In [22]:
def find_alias(params):
    alias = []
    for i in range(len(params)):
        alias.append(params['categories'][i][0]['alias'])
       # if params['categories'][i][0]['alias'] == '':
        #    df.drop(params['categories'][i])
    params['alias'] = alias

In [23]:
find_alias(ssdf)

In [25]:
#Find and destroy empty category
lbdf = lbdf.dropna(subset=['categories'])

In [37]:
#Find and destroy empty list
lbdf = lbdf[lbdf['categories'].map(lambda d: len(d)) > 0]

In [ ]:
#FIND OUT HOW TO KEEP MULTI PURPOSE STORES (SECOND ALIAS)

In [66]:
#Function to filter non-flowershops
def flower_filter(params):
    indexNames = params[(params['alias'] != 'florists') & (params['alias'] != 'floraldesigners') ].index
    params.drop(indexNames, inplace=True)
    return params

In [67]:
lbdf = flower_filter(lbdf)
ssdf = flower_filter(ssdf)

C:\Users\A\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [68]:
ssdf

,name,rating,pricing,num_reviews,street_address,is_closed,review_count,categories,city,distance,longitude,latitude,alias
0,"Bell Flowers, Inc.",5.0,2.0,71,8947 Brookville Rd,False,71,"[{'alias': 'florists', 'title': 'Florists'}]",Silver Spring,2972.202721,-77.052067,39.003009,florists
1,Danisa's Wholesale Fresh Flowers Inc,4.5,2.0,28,8870 Monard Dr,False,28,"[{'alias': 'florists', 'title': 'Florists'}]",Silver Spring,3567.895819,-77.059157,39.001541,florists
2,Potomac Floral Wholesale,3.5,2.0,76,2403 Linden Ln,False,76,"[{'alias': 'florists', 'title': 'Florists'}, {...",Silver Spring,3155.302144,-77.050338,39.009649,florists
4,Garden Vivien,5.0,NaN,1,2223 Kansas Ave,False,1,"[{'alias': 'florists', 'title': 'Florists'}]",Silver Spring,2721.772142,-77.049093,39.003290,florists
5,Da Vinci's Florist,4.0,2.0,37,2756 Garfield Ave,False,37,"[{'alias': 'florists', 'title': 'Florists'}, {...",Silver Spring,3451.436878,-77.057779,39.001881,florists
6,Creative Floral Designs,3.5,2.0,26,12158 Tech Rd,False,26,"[{'alias': 'florists', 'title': 'Florists'}, {...",Silver Spring,7528.587951,-76.967169,39.054732,florists
8,Ella's Florals,5.0,NaN,4,None,False,4,"[{'alias': 'florists', 'title': 'Florists'}]",Silver Spring,5007.226265,-77.028870,38.996680,florists
9,Citiflowers,3.0,NaN,10,2321 Stewart Ave,False,10,"[{'alias': 'florists', 'title': 'Florists'}, {...",Silver Spring,2826.536441,-77.050319,39.002958,florists
10,Roxana Multiservices,3.0,NaN,6,8727 Flower Ave,False,6,"[{'alias': 'floraldesigners', 'title': 'Floral...",Silver Spring,1238.291097,-77.003873,38.999579,floraldesigners
12,Sweet Stems Floral,5.0,NaN,2,3020 Beethoven Way,False,2,"[{'alias': 'floraldesigners', 'title': 'Floral...",Silver Spring,9928.829936,-76.945920,39.069223,floraldesigners


In [69]:
lbdf

,name,rating,pricing,num_reviews,street_address,is_closed,review_count,categories,city,distance,longitude,latitude,alias
0,Loudoun D Floral,4.5,2.0,16,,False,16,"[{'alias': 'florists', 'title': 'Florists'}]",Leesburg,1491.759497,-77.534730,39.130220,florists
1,J Morris Flowers,4.0,3.0,26,120 East Market St,False,26,"[{'alias': 'florists', 'title': 'Florists'}]",Leesburg,1314.094845,-77.561445,39.114299,florists
2,Jerry's Flowers & Gifts,4.0,2.0,22,700 Fieldstone Dr NE,False,22,"[{'alias': 'florists', 'title': 'Florists'}]",Leesburg,1147.831498,-77.545801,39.125919,florists
3,CPK Floral Designs,5.0,NaN,8,None,False,8,"[{'alias': 'florists', 'title': 'Florists'}, {...",Leesburg,1491.759497,-77.531944,39.119907,florists
4,Springvale Floral,5.0,NaN,4,None,False,4,"[{'alias': 'floraldesigners', 'title': 'Floral...",Leesburg,1491.759497,-77.569520,39.112050,floraldesigners
5,House Of Redman,5.0,NaN,1,206 Loudoun St SW,False,1,"[{'alias': 'florists', 'title': 'Florists'}, {...",Leesburg,1858.989002,-77.567840,39.115130,florists
6,Holly Heider Chapple Flowers,4.0,NaN,3,None,False,3,"[{'alias': 'florists', 'title': 'Florists'}]",Leesburg,1491.759497,-77.566540,39.108480,florists
7,Wild Green Yonder,5.0,NaN,2,106 Loudoun St SW,False,2,"[{'alias': 'floraldesigners', 'title': 'Floral...",Leesburg,1773.476827,-77.566845,39.114899,floraldesigners
10,Affordable Arrangements,5.0,NaN,2,,False,2,"[{'alias': 'floraldesigners', 'title': 'Floral...",Leesburg,1491.759497,-77.569405,39.110386,floraldesigners


## Analysis

In [70]:
# Graph totals

In [ ]:
# Retrieve population data from Census

In [ ]:
# Flower shop per 1,000 people

In [ ]:
# Average income from each city

In [ ]:
# Average age from each city

In [ ]:
# Flower buyer demographic info?

## Maps

In [71]:
# Point maps for each city

In [ ]:
# Population heat map

In [ ]:
# Population head map plus point map (show who services who)

## Decision